In [3]:
import numpy as np
import os
from scipy.io import wavfile
from scipy.signal import stft, windows
from sklearn.metrics.pairwise import cosine_similarity

# Função para remover silêncio automaticamente
def remover_silencio(y, threshold=0.02):
    energia = np.abs(y)
    indices = np.where(energia > threshold)[0]
    if len(indices) == 0:
        return y
    return y[indices[0]:indices[-1]]

# Extrai assinatura via SVD
def extrair_assinatura_svd(path_arquivo):
    fs, y = wavfile.read(path_arquivo)
    y = y / np.max(np.abs(y))
    if len(y.shape) > 1:
        y = y[:, 0]
    y = remover_silencio(y)

    n_fft = 2048
    hop_length = n_fft // 2
    window = windows.hann(n_fft)

    if len(y) < n_fft:
        y = np.pad(y, (0, n_fft - len(y)), mode='constant')

    _, _, Zxx = stft(y, fs=fs, window=window, nperseg=n_fft, noverlap=n_fft - hop_length)
    A = np.abs(Zxx)
    U, _, _ = np.linalg.svd(A, full_matrices=False)
    return U[:, 0]

# Agrupa arquivos por nome do acorde (ex: A_maior, C#_menor, etc.)
def agrupar_por_acorde_v2(diretorio_base):
    acordes = {}
    for tipo in os.listdir(diretorio_base):  # "maiores", "menores"
        path_tipo = os.path.join(diretorio_base, tipo)
        if not os.path.isdir(path_tipo):
            continue
        for nota in os.listdir(path_tipo):  # A, B, C, ...
            path_nota = os.path.join(path_tipo, nota)
            if not os.path.isdir(path_nota):
                continue
            chave = f"{nota}_{tipo[:-1]}"  # Ex: A_maior, C_menor
            acordes[chave] = []
            for subpasta in os.listdir(path_nota):  # fundamental, inversao1, inversao2
                path_sub = os.path.join(path_nota, subpasta)
                if not os.path.isdir(path_sub):
                    continue
                for f in os.listdir(path_sub):
                    if f.endswith(".wav"):
                        acordes[chave].append(os.path.join(path_sub, f))
    return acordes

# Calcula a assinatura média de cada classe
def gerar_assinatura_media(lista_caminhos):
    assinaturas = []
    for caminho in lista_caminhos:
        try:
            assinatura = extrair_assinatura_svd(caminho)
            assinaturas.append(assinatura)
            print(f"✔ Assinatura extraída de {os.path.basename(caminho)}")
        except Exception as e:
            print(f"❌ Erro em {caminho}: {e}")
    return np.mean(assinaturas, axis=0)

# Classifica com base na similaridade cosseno
def classificar(vetor_teste, assinaturas):
    max_sim = -1
    classe = None
    for nome_acorde, assinatura in assinaturas.items():
        sim = cosine_similarity([vetor_teste], [assinatura])[0, 0]
        print(f"Acorde {nome_acorde}: Similaridade = {sim:.4f}")
        if sim > max_sim:
            max_sim = sim
            classe = nome_acorde
    return classe

# ========== EXECUÇÃO PRINCIPAL ==========
diretorio_base = "acordes"  # <-- Altere para o caminho certo se necessário
acordes_gravados = agrupar_por_acorde_v2(diretorio_base)

assinaturas_media = {}
for acorde, arquivos in acordes_gravados.items():
    print(f"\n🔍 Processando acorde {acorde}...")
    assinatura = gerar_assinatura_media(arquivos)
    assinaturas_media[acorde] = assinatura


🔍 Processando acorde Do_maior...
✔ Assinatura extraída de C_fundamental_1.wav
✔ Assinatura extraída de C_fundamental_2.wav
✔ Assinatura extraída de C_fundamental_3.wav
✔ Assinatura extraída de C_major.wav
✔ Assinatura extraída de C_inversao1_1.wav
✔ Assinatura extraída de C_inversao1_2.wav
✔ Assinatura extraída de C_inversao1_3.wav
✔ Assinatura extraída de C_inversao2_1.wav
✔ Assinatura extraída de C_inversao2_2.wav
✔ Assinatura extraída de C_inversao2_3.wav

🔍 Processando acorde Fa_maior...
✔ Assinatura extraída de F_fundamental_1.wav
✔ Assinatura extraída de F_fundamental_2.wav
✔ Assinatura extraída de F_fundamental_3.wav
✔ Assinatura extraída de F_major.wav
✔ Assinatura extraída de F_inversao1_1.wav
✔ Assinatura extraída de F_inversao1_2.wav
✔ Assinatura extraída de F_inversao1_3.wav
✔ Assinatura extraída de F_inversao2_1.wav
✔ Assinatura extraída de F_inversao2_2.wav
✔ Assinatura extraída de F_inversao2_3.wav

🔍 Processando acorde La_maior...
✔ Assinatura extraída de A_fundamental

In [5]:
# Classificando um arquivo de teste
arquivo_teste = "audios/acorde_teste2.wav"  
vetor_teste = extrair_assinatura_svd(arquivo_teste)

print("\n📊 Similaridades com acordes conhecidos:")
acorde_predito = classificar(vetor_teste, assinaturas_media)
print(f"\n🎵 Acorde classificado: {acorde_predito}")


📊 Similaridades com acordes conhecidos:
Acorde Do_maior: Similaridade = 0.4460
Acorde Fa_maior: Similaridade = 0.3979
Acorde La_maior: Similaridade = 0.4139
Acorde Mi_maior: Similaridade = 0.3312
Acorde Re_maior: Similaridade = 0.2371
Acorde Si_maior: Similaridade = 0.1854
Acorde Sol_maior: Similaridade = 0.1714
Acorde Do_menor: Similaridade = 0.3374
Acorde Fa_menor: Similaridade = 0.3775
Acorde La_menor: Similaridade = 0.5098
Acorde Mi_menor: Similaridade = 0.2703
Acorde Re_menor: Similaridade = 0.2550
Acorde Si_menor: Similaridade = 0.1820
Acorde Sol_menor: Similaridade = 0.1535

🎵 Acorde classificado: La_menor
